In [1]:
import numpy as np
import pandas as pd
import requests as req
import time
import re
import glob
import os
import subprocess
from bs4 import BeautifulSoup

In [2]:
### Download Smart Contract Meta Data
if False:
    MAX_PAGE = 1000
    AT_PAGE = 0
    contracts = {"address": [], "block_number": [], "ether": [], "has_source": [] }
    base_url = "https://contract-library.com/api/contracts?"
    for idx in range(MAX_PAGE - AT_PAGE):
        ### Start Downloading
        page_number = idx + 1 + AT_PAGE
        end_point = "{0}n=Ethereum&q=&t=address&s=block_number&o=desc&p={1}&c=100&w=".format(base_url, page_number)
        r = req.get(end_point)
        assert(r.status_code == 200)
        print("=> page_number: {}".format(page_number))
        json = r.json()
        for contract in json["contracts"]:
            contracts["address"].append(contract["address"])
            contracts["block_number"].append(contract["block_number"])
            contracts["ether"].append(contract["ether"])
            contracts["has_source"].append(contract["has_source"])
        ## Write Fragment
        if page_number % 100 == 0:
            csv_file = "assets/addr_{}.csv".format(int(page_number / 100))
            df = pd.DataFrame(contracts)
            df.to_csv(csv_file, header=None, index=False)
            print("=> write to {}".format(csv_file))
            print("=> sleep 2 seconds")
            time.sleep(2)
            ### Clear Contract Data
            contracts["address"] = []
            contracts["block_number"] = []
            contracts["ether"] = []
            contracts["has_source"] = []

In [3]:
## Supported compiler versions
if True:
    versions = []
    for i in range(25 - 18 + 1):
        versions.append("0.4.{}".format(i + 18))
    for i in range(16 - 0 + 1):
        versions.append("0.5.{}".format(i))
    for i in range(3 + 1):
        versions.append("0.6.{}".format(i))

In [4]:
print("=> supported version: {}".format(len(versions)))

=> supported version: 29


In [5]:
### Filter contract source code by version
if False:
    df = pd.read_csv(
        "assets/addr.csv",
        header=None,
        names=["address", "block_number", "ether", "has_source"],
    )
    df_with_source = df[(df.has_source == True)]
    print(df_with_source.describe())
    for address in df_with_source["address"]:
        end_point = "https://etherscan.io/address/{}#code".format(address)
        r = req.get(end_point, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"})
        assert(r.status_code == 200)
        print("=> address: {}".format(address))
        soup = BeautifulSoup(r.text, 'html.parser')
        editor = soup.find(id="editor")
        match = re.findall(r"pragma(\s+)solidity(\s+)[^\d]*((\d+\.?)+)", editor.text)
        print(match)
        if len(match) > 0 and len(match[0]) > 2:
            version = match[0][2]
            if version in versions:
                with open("contracts/{}_{}".format(address, version), "w") as f:
                    f.write(editor.text)

In [6]:
### Report 1
if True:
    num_addr = !cat assets/addr.csv | wc -l
    num_addr = int(num_addr[0])
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    print("=> contracts {}".format(num_addr))
    print("=> sup_contracts {}".format(len(files)))
    print("=> % {}".format(len(files) / num_addr * 100))

=> contracts 100000
=> sup_contracts 1982
=> % 1.982


In [7]:
### Filter smart contract if it can not be compiled 
if False:
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    error_files = []
    for idx, file in enumerate(files):
        version = file.split('_')[1]
        os.environ["SYM_SOLC_VERSION"] = version
        os.environ["SYM_FILE"] = file
        !solc use "$SYM_SOLC_VERSION" > /dev/null 2>&1
        output = !solc $SYM_FILE
        if "Error" in ",".join(output):
            print("=> FAIL {}".format(file))
            error_files.append(file)
        else:
            print("=> OK {}".format(file))
    df = pd.DataFrame(error_files)
    csv_file = "contracts/errors.csv"
    df.to_csv(csv_file, header=None, index=False)

In [8]:
### Report 2
if True:
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    num_errors = !cat contracts/errors.csv | wc -l
    num_errors = int(num_errors[0])
    print("=> sup_contracts: {}".format(len(files)))
    print("=> error: {}".format(num_errors))
    print("=> comp : {}".format(len(files) - num_errors))
    print("=> % {}".format((len(files) - num_errors)/len(files) * 100))

=> sup_contracts: 1982
=> error: 231
=> comp : 1751
=> % 88.34510595358223


In [9]:
if False:
    files = !find contracts/* -not -name "*.csv" -not -name "*.json"
    df = pd.read_csv("contracts/errors.csv", header=None, names=["contract_path"])
    for contract_path in df["contract_path"]:
        files.remove(contract_path)
    print("=> comp: {}".format(len(files)))
    compiled_files = []
    for idx, file in enumerate(files):
        version = file.split('_')[1]
        os.environ["SYM_SOLC_VERSION"] = version
        os.environ["SYM_FILE"] = file
        os.environ["SYM_FILE_OUTPUT"] = "{}.json".format(file)
        print("=> {} / {} {}".format(idx + 1, len(files), file))
        !solc use $SYM_SOLC_VERSION > /dev/null 2>&1
        !solc --combined-json bin-runtime,srcmap-runtime,ast $SYM_FILE > $SYM_FILE_OUTPUT
        compiled_files.append(file)
    df = pd.DataFrame(compiled_files)
    csv_file = "contracts/compiled.csv"
    df.to_csv(csv_file, header=None, index=False)

In [10]:
### Report 3
if True:
    files = !find contracts/*.json
    num_files = !cat contracts/compiled.csv | wc -l
    print("=> json {}".format(len(files)))
    print("=> num_files {}".format(int(num_files[0])))

=> json 1750
=> num_files 1750


In [11]:
loop_bound = 20

env_content = """
dataload=02
expectCoverage=0.1
maxVisitedBlock={}
maxVisitedBlockBound=50
maxVisitedBlockStep=2
allocatedRange="a0"
""".strip().format(loop_bound)

In [12]:
### Run symEvm for coverage
if False:
    result = {
        "address": [], 
        "contract_name": [],
        "endpoints": [], 
        "covered_jumpis": [], 
        "total_jumpis": [],
        "duration": [],
        "bytelen": [],
        "loop_bound": [],
    }
    df = pd.read_csv("contracts/compiled.csv", header=None, names=["contract_path"])
    coverage_path = "results/coverage.csv"
    with open(".env", "w") as f:
        f.write(env_content)
    if os.path.exists(coverage_path):
         os.remove(coverage_path)
    with open(coverage_path, "w") as f:
        pass
    contract_paths = df["contract_path"]
    for idx, contract_path in enumerate(contract_paths):
        os.environ["SYM_CONTRACT_FILE"] = contract_path
        os.environ["SYM_JSON_FILE"] = "{}.json".format(contract_path)
        os.environ["SYM_STEP"] = "1"
        print("{} / {}".format(idx + 1, len(contract_paths)))
        ## if json file is empty
        with open(contract_path + ".json", "r") as f:
            if len(f.read()) == 0:
                continue
        ## execute SymEvm
        addr = contract_path.split("/")[1].split("_")[0]
        start_time = time.time()
        output = !node index.js
        duration = time.time() - start_time
        output = '\n'.join(output)
        print("addr: {}".format(addr))
        print(output)
        ## contract
        if "error" not in output:
            match = re.findall(r"Start Analyzing Contract:\s+([^\n]+)", output)
            assert(len(match) > 0)
            contract_name = match[0]
            ## endpoints
            match = re.findall(r"endpoints\s+:\s+(\d+)", output)
            assert(len(match) > 0)
            endpoints = int(match[0])
            ## cjumpis
            match = re.findall(r"cjumpis\s+:\s+(\d+)", output)
            assert(len(match) > 0)
            cjumpis = int(match[0])
            ## njumpis
            match = re.findall(r"njumpis\s+:\s+(\d+)", output)
            assert(len(match) > 0)
            njumpis = int(match[0])
            ## byte len
            match = re.findall(r"bytelen\s+:\s+(\d+)", output)
            assert(len(match) > 0)
            bytelen = int(match[0])
            ##
            result["address"].append(addr)
            result["endpoints"].append(endpoints)
            result["covered_jumpis"].append(cjumpis)
            result["total_jumpis"].append(njumpis)
            result["contract_name"].append(contract_name)
            result["duration"].append(round(duration * 1000))
            result["bytelen"].append(bytelen)
            result["loop_bound"].append(loop_bound)
        
    df = pd.DataFrame(result)
    df.to_csv(coverage_path, index=None)
    !rm report.*

In [13]:
loop_bound = 20

env_content = """
dataload=02
expectCoverage=0.1
maxVisitedBlock={}
maxVisitedBlockBound=50
maxVisitedBlockStep=2
allocatedRange="a0"
""".strip().format(loop_bound)

In [14]:
### Run symEvm for tainting
if False:
    result = {
        "address": [], 
        "success_sloads": [],
        "success_mloads": [],
        "success_mstores": [],
        "success_sstores": [],
        "failed_sloads": [],
        "failed_mloads": [],
        "failed_mstores": [],
        "failed_sstores": [],
    }
    df = pd.read_csv("contracts/compiled.csv", header=None, names=["contract_path"])
    tainting_path = "results/tainting.csv"
    with open(".env", "w") as f:
        f.write(env_content)
    if os.path.exists(tainting_path):
         os.remove(tainting_path)
    with open(tainting_path, "w") as f:
        pass
    contract_paths = df["contract_path"]
    for idx, contract_path in enumerate(contract_paths):
        os.environ["SYM_CONTRACT_FILE"] = contract_path
        os.environ["SYM_JSON_FILE"] = "{}.json".format(contract_path)
        os.environ["SYM_STEP"] = "2"
        print("{} / {}".format(idx + 1, len(contract_paths)))
        ## if json file is empty
        with open(contract_path + ".json", "r") as f:
            if len(f.read()) == 0:
                continue
        ## execute SymEvm
        addr = contract_path.split("/")[1].split("_")[0]
        output = !node index.js
        output = '\n'.join(output)
        print("addr: {}".format(addr))
        print(output)
        if "error" not in output:
            ## success
            match = re.findall(r"success\s+:\s+\[([^\]]+)", output)
            assert(len(match) > 0)
            success = list(map(lambda x: int(x), match[0].split(',')))
            ## failed
            match = re.findall(r"failed\s+:\s+\[([^\]]+)", output)
            assert(len(match) > 0)
            failed = list(map(lambda x: int(x), match[0].split(',')))
            ##
            result["address"].append(addr)
            result["success_sloads"].append(success[0])
            result["success_mloads"].append(success[1])
            result["success_mstores"].append(success[2])
            result["success_sstores"].append(success[3])
            result["failed_sloads"].append(failed[0])
            result["failed_mloads"].append(failed[1])
            result["failed_mstores"].append(failed[2])
            result["failed_sstores"].append(failed[3])
        
    df = pd.DataFrame(result)
    df.to_csv(tainting_path, index=None)
    !rm report.*

In [17]:
loop_bound = 20

env_content = """
dataload=02
expectCoverage=0.95
maxVisitedBlock={}
maxVisitedBlockBound=50
maxVisitedBlockStep=10
allocatedRange="a0"
""".strip().format(loop_bound)

In [25]:
### Run symEvm for tainting
if True:
    result = {
        "address": [],
        "integer": [],
        "disorder": [],
        "frez": [],
        "reentrancy": [],
    }
    df = pd.read_csv("contracts/compiled.csv", header=None, names=["contract_path"])
    bugs_path = "results/bugs.csv"
    with open(".env", "w") as f:
        f.write(env_content)
    if os.path.exists(bugs_path):
         os.remove(bugs_path)
    with open(bugs_path, "w") as f:
        pass
    contract_paths = df["contract_path"][8:9]
    for idx, contract_path in enumerate(contract_paths):
        os.environ["SYM_CONTRACT_FILE"] = contract_path
        os.environ["SYM_JSON_FILE"] = "{}.json".format(contract_path)
        os.environ["SYM_STEP"] = "3"
        print(contract_path)
        print("{} / {}".format(idx + 1, len(contract_paths)))
        ## if json file is empty
        with open(contract_path + ".json", "r") as f:
            if len(f.read()) == 0:
                continue
        ## execute SymEvm
        addr = contract_path.split("/")[1].split("_")[0]
        !node index.js
#         output = '\n'.join(output)
#         print(output)
#         if "error" not in output:
#             ## integer
#             match = re.findall(r"integer\s+:\s+(false|true)", output)
#             assert(len(match) > 0)
#             result["integer"].append(0 if match[0] == 'false' else 1)
#             ## disorder
#             match = re.findall(r"disorder\s+:\s+(false|true)", output)
#             assert(len(match) > 0)
#             result["disorder"].append(0 if match[0] == 'false' else 1)
#             ## frez
#             match = re.findall(r"frez\s+:\s+(false|true)", output)
#             assert(len(match) > 0)
#             result["frez"].append(0 if match[0] == 'false' else 1)
#             ## reentrancy
#             match = re.findall(r"reentrancy\s+:\s+(false|true)", output)
#             assert(len(match) > 0)
#             result["reentrancy"].append(0 if match[0] == 'false' else 1)
#             result["address"].append(addr)

    df = pd.DataFrame(result)
    df.to_csv(bugs_path, index=None)

contracts/0x0162c142bf923d00ec510c3c7e7a4072194d291a_0.5.7
1 / 1
[symEvm] info: Start Analyzing Contract: AletheaNoLoss
[symEvm] info: update maxVisitedBlock to 30
[symEvm] info: update maxVisitedBlock to 40
[symEvm] info: update maxVisitedBlock to 50
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 2664
[symEvm] info: |	cjumpis    : 157
[symEvm] info: |	njumpis    : 157
[symEvm] info: |	bytelen    : 14186
endPointIdx: 0
endPointIdx: 1
endPointIdx: 2
endPointIdx: 3
endPointIdx: 4
endPointIdx: 5
endPointIdx: 6
endPointIdx: 7
endPointIdx: 8
endPointIdx: 9
endPointIdx: 10
endPointIdx: 11
endPointIdx: 12
endPointIdx: 13
endPointIdx: 14
endPointIdx: 15
endPointIdx: 16
endPointIdx: 17
endPointIdx: 18
endPointIdx: 19
endPointIdx: 20
endPointIdx: 21
endPointIdx: 22
endPointIdx: 23
endPointIdx: 24
endPointIdx: 25
endPointIdx: 26
endPointIdx: 27
endPointIdx: 28
endPointIdx: 29
endPointIdx: 30
endPointIdx: 31
endPointIdx: 32
endPointIdx: 33
endPointIdx: 

endPointIdx: 452
endPointIdx: 453
endPointIdx: 454
endPointIdx: 455
endPointIdx: 456
endPointIdx: 457
endPointIdx: 458
endPointIdx: 459
endPointIdx: 460
endPointIdx: 461
endPointIdx: 462
endPointIdx: 463
endPointIdx: 464
endPointIdx: 465
endPointIdx: 466
endPointIdx: 467
endPointIdx: 468
endPointIdx: 469
endPointIdx: 470
endPointIdx: 471
endPointIdx: 472
endPointIdx: 473
endPointIdx: 474
endPointIdx: 475
endPointIdx: 476
endPointIdx: 477
endPointIdx: 478
endPointIdx: 479
endPointIdx: 480
endPointIdx: 481
endPointIdx: 482
endPointIdx: 483
endPointIdx: 484
endPointIdx: 485
endPointIdx: 486
endPointIdx: 487
endPointIdx: 488
endPointIdx: 489
endPointIdx: 490
endPointIdx: 491
endPointIdx: 492
endPointIdx: 493
endPointIdx: 494
endPointIdx: 495
endPointIdx: 496
endPointIdx: 497
endPointIdx: 498
endPointIdx: 499
endPointIdx: 500
endPointIdx: 501
endPointIdx: 502
endPointIdx: 503
endPointIdx: 504
endPointIdx: 505
endPointIdx: 506
endPointIdx: 507
endPointIdx: 508
endPointIdx: 509
endPointIdx: 5

endPointIdx: 934
endPointIdx: 935
endPointIdx: 936
endPointIdx: 937
endPointIdx: 938
endPointIdx: 939
endPointIdx: 940
endPointIdx: 941
endPointIdx: 942
endPointIdx: 943
endPointIdx: 944
endPointIdx: 945
endPointIdx: 946
endPointIdx: 947
endPointIdx: 948
endPointIdx: 949
endPointIdx: 950
endPointIdx: 951
endPointIdx: 952
endPointIdx: 953
endPointIdx: 954
endPointIdx: 955
endPointIdx: 956
endPointIdx: 957
endPointIdx: 958
endPointIdx: 959
endPointIdx: 960
endPointIdx: 961
endPointIdx: 962
endPointIdx: 963
endPointIdx: 964
endPointIdx: 965
endPointIdx: 966
endPointIdx: 967
endPointIdx: 968
endPointIdx: 969
endPointIdx: 970
endPointIdx: 971
endPointIdx: 972
endPointIdx: 973
endPointIdx: 974
endPointIdx: 975
endPointIdx: 976
endPointIdx: 977
endPointIdx: 978
endPointIdx: 979
endPointIdx: 980
endPointIdx: 981
endPointIdx: 982
endPointIdx: 983
endPointIdx: 984
endPointIdx: 985
endPointIdx: 986
endPointIdx: 987
endPointIdx: 988
endPointIdx: 989
endPointIdx: 990
endPointIdx: 991
endPointIdx: 9

endPointIdx: 1393
endPointIdx: 1394
endPointIdx: 1395
endPointIdx: 1396
endPointIdx: 1397
endPointIdx: 1398
endPointIdx: 1399
endPointIdx: 1400
endPointIdx: 1401
endPointIdx: 1402
endPointIdx: 1403
endPointIdx: 1404
endPointIdx: 1405
endPointIdx: 1406
endPointIdx: 1407
endPointIdx: 1408
endPointIdx: 1409
endPointIdx: 1410
endPointIdx: 1411
endPointIdx: 1412
endPointIdx: 1413
endPointIdx: 1414
endPointIdx: 1415
endPointIdx: 1416
endPointIdx: 1417
endPointIdx: 1418
endPointIdx: 1419
endPointIdx: 1420
endPointIdx: 1421
endPointIdx: 1422
endPointIdx: 1423
endPointIdx: 1424
endPointIdx: 1425
endPointIdx: 1426
endPointIdx: 1427
endPointIdx: 1428
endPointIdx: 1429
endPointIdx: 1430
endPointIdx: 1431
endPointIdx: 1432
endPointIdx: 1433
endPointIdx: 1434
endPointIdx: 1435
endPointIdx: 1436
endPointIdx: 1437
endPointIdx: 1438
endPointIdx: 1439
endPointIdx: 1440
endPointIdx: 1441
endPointIdx: 1442
endPointIdx: 1443
endPointIdx: 1444
endPointIdx: 1445
endPointIdx: 1446
endPointIdx: 1447
endPointId

endPointIdx: 1849
endPointIdx: 1850
endPointIdx: 1851
endPointIdx: 1852
endPointIdx: 1853
endPointIdx: 1854
endPointIdx: 1855
endPointIdx: 1856
endPointIdx: 1857
endPointIdx: 1858
endPointIdx: 1859
endPointIdx: 1860
endPointIdx: 1861
endPointIdx: 1862
endPointIdx: 1863
endPointIdx: 1864
endPointIdx: 1865
endPointIdx: 1866
endPointIdx: 1867
endPointIdx: 1868
endPointIdx: 1869
endPointIdx: 1870
endPointIdx: 1871
endPointIdx: 1872
endPointIdx: 1873
endPointIdx: 1874
endPointIdx: 1875
endPointIdx: 1876
endPointIdx: 1877
endPointIdx: 1878
endPointIdx: 1879
endPointIdx: 1880
endPointIdx: 1881
endPointIdx: 1882
endPointIdx: 1883
endPointIdx: 1884
endPointIdx: 1885
endPointIdx: 1886
endPointIdx: 1887
endPointIdx: 1888
endPointIdx: 1889
endPointIdx: 1890
endPointIdx: 1891
endPointIdx: 1892
endPointIdx: 1893
endPointIdx: 1894
endPointIdx: 1895
endPointIdx: 1896
endPointIdx: 1897
endPointIdx: 1898
endPointIdx: 1899
endPointIdx: 1900
endPointIdx: 1901
endPointIdx: 1902
endPointIdx: 1903
endPointId

endPointIdx: 2306
endPointIdx: 2307
endPointIdx: 2308
endPointIdx: 2309
endPointIdx: 2310
endPointIdx: 2311
endPointIdx: 2312
endPointIdx: 2313
endPointIdx: 2314
endPointIdx: 2315
endPointIdx: 2316
endPointIdx: 2317
endPointIdx: 2318
endPointIdx: 2319
endPointIdx: 2320
endPointIdx: 2321
endPointIdx: 2322
endPointIdx: 2323
endPointIdx: 2324
endPointIdx: 2325
endPointIdx: 2326
endPointIdx: 2327
endPointIdx: 2328
endPointIdx: 2329
endPointIdx: 2330
endPointIdx: 2331
endPointIdx: 2332
endPointIdx: 2333
endPointIdx: 2334
endPointIdx: 2335
endPointIdx: 2336
endPointIdx: 2337
endPointIdx: 2338
endPointIdx: 2339
endPointIdx: 2340
endPointIdx: 2341
endPointIdx: 2342
endPointIdx: 2343
endPointIdx: 2344
endPointIdx: 2345
endPointIdx: 2346
endPointIdx: 2347
endPointIdx: 2348
endPointIdx: 2349
endPointIdx: 2350
endPointIdx: 2351
endPointIdx: 2352
endPointIdx: 2353
endPointIdx: 2354
endPointIdx: 2355
endPointIdx: 2356
endPointIdx: 2357
endPointIdx: 2358
endPointIdx: 2359
endPointIdx: 2360
endPointId

reentrancy: 1318
reentrancy: 1319
reentrancy: 1320
reentrancy: 1321
reentrancy: 1322
reentrancy: 1323
reentrancy: 1324
reentrancy: 1325
reentrancy: 1326
reentrancy: 1327
reentrancy: 1328
reentrancy: 1329
reentrancy: 1330
reentrancy: 1331
reentrancy: 1332
reentrancy: 1333
reentrancy: 1334
reentrancy: 1335
reentrancy: 1336
reentrancy: 1337
reentrancy: 1338
reentrancy: 1339
reentrancy: 1340
reentrancy: 1341
reentrancy: 1342
reentrancy: 1343
reentrancy: 1344
reentrancy: 1345
reentrancy: 1346
reentrancy: 1347
reentrancy: 1348
reentrancy: 1349
reentrancy: 1350
reentrancy: 1351
reentrancy: 1352
reentrancy: 1353
reentrancy: 1354
reentrancy: 1355
reentrancy: 1356
reentrancy: 1357
reentrancy: 1358
reentrancy: 1359
reentrancy: 1360
reentrancy: 1361
reentrancy: 1362
reentrancy: 1363
reentrancy: 1364
reentrancy: 1365
reentrancy: 1366
reentrancy: 1367
reentrancy: 1368
reentrancy: 1369
reentrancy: 1370
reentrancy: 1371
reentrancy: 1372
reentrancy: 1373
reentrancy: 1374
reentrancy: 1375
reentrancy: 13

reentrancy: 1988
reentrancy: 1989
reentrancy: 1990
reentrancy: 1991
reentrancy: 1992
reentrancy: 1993
reentrancy: 1994
reentrancy: 1995
reentrancy: 1996
reentrancy: 1997
reentrancy: 1998
reentrancy: 1999
reentrancy: 2000
reentrancy: 2001
reentrancy: 2002
reentrancy: 2003
reentrancy: 2004
reentrancy: 2005
reentrancy: 2006
reentrancy: 2007
reentrancy: 2008
reentrancy: 2009
reentrancy: 2010
reentrancy: 2011
reentrancy: 2012
reentrancy: 2013
reentrancy: 2014
reentrancy: 2015
reentrancy: 2016
reentrancy: 2017
reentrancy: 2018
reentrancy: 2019
reentrancy: 2020
reentrancy: 2021
reentrancy: 2022
reentrancy: 2023
reentrancy: 2024
reentrancy: 2025
reentrancy: 2026
reentrancy: 2027
reentrancy: 2028
reentrancy: 2029
reentrancy: 2030
reentrancy: 2031
reentrancy: 2032
reentrancy: 2033
reentrancy: 2034
reentrancy: 2035
reentrancy: 2036
reentrancy: 2037
reentrancy: 2038
reentrancy: 2039
reentrancy: 2040
reentrancy: 2041
reentrancy: 2042
reentrancy: 2043
reentrancy: 2044
reentrancy: 2045
reentrancy: 20

reentrancy: 2489
reentrancy: 2490
reentrancy: 2491
reentrancy: 2492
reentrancy: 2493
reentrancy: 2494
reentrancy: 2495
reentrancy: 2496
reentrancy: 2497
reentrancy: 2498
reentrancy: 2499
reentrancy: 2500
reentrancy: 2501
reentrancy: 2502
reentrancy: 2503
reentrancy: 2504
reentrancy: 2505
reentrancy: 2506
reentrancy: 2507
reentrancy: 2508
reentrancy: 2509
reentrancy: 2510
reentrancy: 2511
reentrancy: 2512
reentrancy: 2513
reentrancy: 2514
reentrancy: 2515
reentrancy: 2516
reentrancy: 2517
reentrancy: 2518
reentrancy: 2519
reentrancy: 2520
reentrancy: 2521
reentrancy: 2522
reentrancy: 2523
reentrancy: 2524
reentrancy: 2525
reentrancy: 2526
reentrancy: 2527
reentrancy: 2528
reentrancy: 2529
reentrancy: 2530
reentrancy: 2531
reentrancy: 2532
reentrancy: 2533
reentrancy: 2534
reentrancy: 2535
reentrancy: 2536
reentrancy: 2537
reentrancy: 2538
reentrancy: 2539
reentrancy: 2540
reentrancy: 2541
reentrancy: 2542
reentrancy: 2543
reentrancy: 2544
reentrancy: 2545
reentrancy: 2546
reentrancy: 25